In [2]:
from VAMPIRE_backend import bdreg,pca_bdreg,clusterSM

In [3]:
from __future__ import division
import pandas as pd
pd.options.mode.chained_assignment = None
import os
import pickle
import numpy as np
from matplotlib import pyplot as plt
from natsort import natsorted

In [4]:
from time import time
def _time(f):
    def wrapper(*args,**kwargs):
        start=time()
        r=f(*args,**kwargs)
        end=time()
        print("%s timed %.3f" %(f.__name__,end-start))
        return r
    return wrapper

In [4]:
bdreg = _time(bdreg)
pca_bdreg = _time(pca_bdreg)
clusterSM = _time(clusterSM)

In [5]:
N=50 #equidistant points along cell boundary

dfsrc = r'\\fatherserverdw\Q\research\images\skin_aging\wsi\hovernet_out\df'
outpth = os.path.join(os.path.dirname(dfsrc),'vampire_out')

In [6]:
setpaths = natsorted([_ for _ in os.listdir(dfsrc) if _.endswith('pkl')])
setIDs = [os.path.splitext(_)[0] for _ in setpaths] #name of plot?
conditions = setIDs #clear definition is needed in terms of how this will be used later
setpaths = [os.path.join(dfsrc,_) for _ in setpaths]

In [7]:
setpaths = [_ for _ in setpaths if not 'duplicate' in _]

In [8]:
# iterate dfs to compile big df
# pkls = pd.read_pickle('2d_skin_hovernet.pkl')
start = time()
pkls = []
for setpath in setpaths:
    pkl = pd.read_pickle(setpath)
    pkls.append(pkl)
pkls = pd.concat(pkls,ignore_index=True)
print(round(time()-start),'seconds')

170 seconds


In [11]:
pkls['imID'] = pkls['imID'].astype(int)

In [12]:
start = time()
pkls.to_feather('2d_skin_hovernet.ftr')
print(round(time()-start),'seconds')

159 seconds


In [13]:
start = time()
pkls.to_pickle('2d_skin_hovernet.pkl')
print(round(time()-start),'seconds')

295 seconds


In [5]:
start = time()
pkls2 =pd.read_feather('2d_skin_hovernet.ftr') #42 sec
print(round(time()-start),'seconds')

42 seconds


In [15]:

start = time()
pkls2 = pd.read_pickle('2d_skin_hovernet.pkl')
print(round(time()-start),'seconds')

MemoryError: 

In [ ]:
# visualize cell distribution
numclass = 12
a=plt.hist(pkls['type'],numclass)
a[0]

In [6]:
pkls2.head( )

,bbox,centroid,contour,type,Area,AR,Sol,Extent,EquiDia,imID
0,"[[1627, 12118], [1646, 12127]]","[1881.754716981132, 11877.27358490566]","[[1886, 11867], [1885, 11868], [1884, 11868], ...",1,84.0,0.481259,0.796209,0.491228,10.341766,8
1,"[[3740, 8056], [3749, 8071]]","[3966.7162162162163, 7840.310810810811]","[[3970, 7836], [3969, 7837], [3968, 7837], [39...",2,56.5,0.364545,0.824818,0.418519,8.481629,8
2,"[[3747, 8029], [3768, 8047]]","[3941.6615384615384, 7852.626923076923]","[[3940, 7843], [3939, 7844], [3938, 7844], [39...",2,231.5,0.770438,0.920477,0.612434,17.168429,8
3,"[[3748, 8045], [3760, 8058]]","[3954.541666666667, 7849.21875]","[[3954, 7844], [3953, 7845], [3952, 7845], [39...",2,79.5,0.662782,0.893258,0.509615,10.060941,8
4,"[[3751, 8061], [3763, 8068]]","[3967.890625, 7852.359375]","[[3969, 7847], [3968, 7848], [3966, 7848], [39...",2,49.5,0.620038,0.916667,0.589286,7.938851,8


In [ ]:
start = time()
pkls2.to_hdf('2d_skin_hovernet.h5', key='df', mode='w')
print(round(time()-start),'seconds')

In [ ]:
1+2

In [ ]:
comps = []
for value in pkls.imID.unique():
    numclass = 12
    a=plt.hist(pkls.loc[pkls['imID'] == value]['type'],numclass)
    comps.append(a[0]/sum(a[0])*100)

In [ ]:
compsdf = pd.DataFrame(comps)
compsdf['imID']= pkls.imID.unique()
compsdf.to_csv('2d_skin_cell_comp.csv')

In [ ]:
#build model by cell type
for clnum in range(2,9):
    modelname = 'sweat_cl{}'.format(clnum)
    idx = 6
    bd1 = pkls[pkls['type']==idx+1].reset_index()
    B=bd1['contour']
    VamModel = {
                "N": [],
                "bdrn": [],
                "mdd": [],
                "pc": [],
                "clnum": [],
                "pcnum": [],
                "mincms": [],
                "testmean": [],
                "teststd": [],
                "boxcoxlambda": [],
                "C": [],
                "Z": []
            }
    build_model = True
    bdpc, VamModel = bdreg(B, N, VamModel, build_model)
    score, VamModel = pca_bdreg(bdpc, VamModel, True)
    pcnum = None # none is 20 by default
    IDX, IDX_dist, VamModel, goodness = clusterSM(outpth, score, bdpc, clnum, pcnum, VamModel, True, None, None,modelname)

    f = open(os.path.join(*[outpth, modelname, modelname+'.pickle']), 'wb')
    pickle.dump(VamModel, f)
    f.close()

In [ ]:
#build model by cell type
tissues = ['corneum','spinosum','shaft','follicle','muscle','oil','sweat','nerve','blood','ecm','fat','white']
for clnum in range(2,9):
    for idx,tissue in enumerate(tissues):
        if tissue=='corneum':continue
        modelname = '{}_cl{}'.format(tissue,clnum)
        bd1 = pkls[pkls['type']==idx+1].reset_index()
        B=bd1['contour']
        VamModel = {
                    "N": [],
                    "bdrn": [],
                    "mdd": [],
                    "pc": [],
                    "clnum": [],
                    "pcnum": [],
                    "mincms": [],
                    "testmean": [],
                    "teststd": [],
                    "boxcoxlambda": [],
                    "C": [],
                    "Z": []
                }
        build_model = True
        bdpc, VamModel = bdreg(B, N, VamModel, build_model)
        score, VamModel = pca_bdreg(bdpc, VamModel, True)
        pcnum = None # none is 20 by default
        IDX, IDX_dist, VamModel, goodness = clusterSM(outpth, score, bdpc, clnum, pcnum, VamModel, True, None, None,modelname)

        f = open(os.path.join(*[outpth, modelname, modelname+'.pickle']), 'wb')
        pickle.dump(VamModel, f)
        f.close()

building model


In [ ]:
clnum = 5
modelname = 'corneum_cl{}'.format(clnum)
modelpth = os.path.join(*[outpth, modelname, modelname+'.pickle'])

In [ ]:
# load model
experimental = False
build_model = False
try:
    f = open(modelpth, 'rb')
except:
    print('the model does not exist. please replace model name to the one you built')
vampire_model = pickle.load(f)
N = vampire_model['N']
clnum = vampire_model['clnum']
pcnum = vampire_model['pcnum']

In [ ]:
dists=np.zeros((len(setpaths),clnum))
for idxA,(setpath,condition,setID) in enumerate(zip(setpaths,conditions,setIDs)):
    pkl = pd.read_pickle(setpath)
    pkl = pkl[pkl['contour'].map(len) > 5]
    pkl = pkl[pkl['type']==idx+1].reset_index(drop=True)
    B = pkl['contour']
    pkl = pkl.drop(columns=['contour'])
    bdpc, vampire_model = bdreg(B, N, vampire_model, build_model) #409sec
    score, vampire_model = pca_bdreg(bdpc, vampire_model, build_model)
    IDX, IDX_dist, VamModel, goodness = clusterSM(outpth, score, bdpc, clnum, pcnum, vampire_model, False, condition, setID,modelname)
    pkl['Shape mode']= pd.Series(IDX)
    pkl.to_csv(setpath.replace('pkl','csv'), index=False)
    a=plt.hist(pkl['Shape mode'],clnum)
    dists[idxA,:] = np.array(a[0])

In [ ]:
dists2 = np.around(dists*100/dists.sum(axis=1)[:,None],decimals=2)
dists2 = pd.DataFrame(dists2)
dists2['setid']=setIDs
sumfn = 'vampire summary.csv'
sumfn = os.path.join(*[outpth, modelname,sumfn])
dists2.to_csv(sumfn)